### Semi-supervised clustering with Infomap
Using a modified map equation to also encode metadata according to [A Map Equation with Metadata: Varying the Role of Attributes in Community Detection](https://arxiv.org/abs/1810.10433) by Emmons 2019.

In [ ]:
import numpy as np
import pandas as pd
import networkx as nx
import infomap
from collections import defaultdict
import matplotlib.colors as colors
import pathlib
import seaborn as sns
import holoviews as hv
import hvplot.networkx as hvnx
import hvplot.pandas
hv.extension('bokeh', 'matplotlib')

In [ ]:
"""
Generate and draw a network with NetworkX, colored
according to the community structure found by Infomap.
"""

def findCommunities(G, flags="", metaDataName="metaData", computeCodelengthOfMetaDataCommunities=False):
    """
    Partition network with the Infomap algorithm.
    Annotates nodes with 'community' id.
    If 'computeCodelengthOfMetaDataCommunities' is True, calculate codelength
    on a partition defined by meta data.
    """

    if computeCodelengthOfMetaDataCommunities:
        flags += " --no-infomap"

    im = infomap.Infomap(flags)

    for (source, target) in G.edges:
        im.add_link(source, target)
    
    metaData = nx.get_node_attributes(G, metaDataName)
    for nodeId, meta in metaData.items():
        im.set_meta_data(nodeId, meta)

    if computeCodelengthOfMetaDataCommunities:
        im.run(initial_partition = { nodeId: int(moduleId) for nodeId, moduleId in nx.get_node_attributes(G, metaDataName).items() })
    else:
        im.run()

    # Store result on nodes and graph
    flow = { node.node_id: node.data.flow for node in im.nodes }
    nx.set_node_attributes(G, metaData, 'meta_data')
    nx.set_node_attributes(G, im.get_modules(), 'modules')
    nx.set_node_attributes(G, flow, 'flow')
    G.graph['num_modules'] = im.num_top_modules
    G.graph['codelength'] = im.codelength
    G.graph['index_codelength'] = im.index_codelength
    G.graph['module_codelength'] = im.module_codelength
    G.graph['meta_codelength'] = im.meta_codelength
    G.graph['meta_entropy'] = im.meta_entropy
    
    return im.num_top_modules, im.codelength, im.meta_codelength, im.meta_entropy


def drawNetwork(G, pos=None, label=None):
    if pos is None:
        pos = nx.spring_layout(G, seed=5)
#     cmap = 'Set3'
    modules = nx.get_node_attributes(G, 'modules')
    num_modules = G.graph['num_modules']
    palette = sns.color_palette("pastel", num_modules)
    cmap = colors.ListedColormap(palette)
    shapes = list('so^>v<d') + ['hex', 'circle_x', 'diamond_cross']
    
    meta_nodelist = defaultdict(list)
    for node, meta in G.nodes(data='meta_data'):
        meta_nodelist[meta].append(node)
    
    if len(meta_nodelist) > len(shapes):
        return hvnx.draw(G, pos, node_color='modules', labels='meta_data', cmap=cmap, label=label)

    edges = hvnx.draw_networkx_edges(G, pos, alpha=0.5)
    nodes = []
    for meta, nodelist in meta_nodelist.items():
        # split on community for different colors
        # TODO: Why doesn't node_shape work with hvnx? Workaround: Use hv.Scatter with marker option
        homogenous_nodes = defaultdict(list)
        for node in nodelist:
            homogenous_nodes[modules[node]].append(node)
        for c, same_nodes in homogenous_nodes.items():
            sub_pos = [pos[node] for node in same_nodes]
            nodes.append(hv.Scatter(sub_pos).options(hv.opts.Scatter(size=20, marker=shapes[meta], color=cmap(c))))
#     labels = hvnx.draw_networkx_nodes(G, pos, labels='metaData', node_alpha=0)
    
    return hv.Overlay([edges, hv.Overlay(nodes)], label=label)

In [ ]:
def addMetaData(G, numCategories=2):
    metaData = {}
    numNodesPerCategory = G.number_of_nodes() // numCategories + 1
    for n in G.nodes:
        metaId = n // numNodesPerCategory
        metaData[n] = metaId
    nx.set_node_attributes(G, values=metaData, name='metaData')

In [ ]:
G = nx.ring_of_cliques(num_cliques=3, clique_size=5)
addMetaData(G, numCategories=3)
metaDataRates=[0, 0.5, 1, 1.5, 2, 3]
graphs = []

for eta in metaDataRates:
    findCommunities(G, flags = f"-2 -N1 --meta-data-rate {eta}")
    M = G.graph['num_modules']
    Hmeta = G.graph['meta_entropy']
    L = G.graph['codelength']
    Lmeta = G.graph['meta_codelength']
    graphs.append(drawNetwork(G, label=f"{M} modules, eta = {eta:.1f}, Hmeta = {Hmeta:.2f}, L = {L:.2f} bits"))
hv.Layout(graphs).cols(2)

Note:
* Meta category encoded by node shape, modules by color.
* Modular boundaries from network structure are still respected (the extra cost from meta data entropies _within_ modules doesn't change that).
* By increasing `eta`, modules with different meta data are eventually segregated to modules with homogenous meta data.

In [ ]:
def plogp(p):
    p = np.asarray(p)
    return p * np.log2(p, where = p>0)

def calc_entropy(p):
    p = np.asarray(p)
    sumP = np.sum(p)
    return -np.sum(plogp(p / sumP)), sumP

def calc_meta_entropy(modules, meta_data, node_flow):
    # module -> (meta -> prob)
    meta_probs = defaultdict(lambda: defaultdict(float))
    for node, module in modules.items():
        meta = meta_data[node]
        flow = node_flow[node]
        meta_probs[module][meta] += flow

    metaEntropy = 0.0
    for module, metaprob in meta_probs.items():
        moduleMetaProbs = list(metaprob.values())
        H, sumProb = calc_entropy(moduleMetaProbs)
        metaEntropy += H * sumProb
    return metaEntropy

eta = 1.0
findCommunities(G, flags = f"-2 -N1 --meta-data-rate {eta}")
M = G.graph['num_modules']
Hmeta = G.graph['meta_entropy']
L = G.graph['codelength']
Lindex = G.graph['index_codelength']
Lmodules = G.graph['module_codelength']
Lmeta = G.graph['meta_codelength']
LwithoutMeta = L - Lmeta
print("Codelength parts and meta data entropy check:")
print(f"Codelength = index codelength ({Lindex}) + module codelength ({Lmodules})")
print(f"Module codelength = {Lmodules - Lmeta} + meta codelength ({Lmeta})")
print(f"Meta codelength = eta ({eta}) * meta entropy ({Hmeta})")
modules = nx.get_node_attributes(G, 'modules')
meta_data = nx.get_node_attributes(G, 'meta_data')
node_flow = nx.get_node_attributes(G, 'flow')
# Compare with externally calculated meta entropy
metaEntropy = calc_meta_entropy(modules, meta_data, node_flow)
print(f"Externally calculated meta entropy: {metaEntropy} bits")

In [ ]:
G = nx.karate_club_graph()
addMetaData(G, numCategories=3)
metaDataRates = np.arange(0, 2, 0.2)
graphs = []
stats = []
for eta in metaDataRates:
    M, L, Lmeta, Hmeta = findCommunities(G, flags = f"-2 --meta-data-rate {eta}")
    graphs.append(drawNetwork(G, label=f"{M} modules, eta = {eta:.1f}, Hmeta = {Hmeta:.2f}, L = {L:.2f} bits"))
    stats.append([eta, M, L, L - Lmeta, Lmeta, Hmeta])
df = pd.DataFrame(stats, columns=['eta', 'Number of modules', 'L', 'L - Lmeta', 'Lmeta = eta * Hmeta', 'Hmeta'])
df.set_index('eta', inplace=True)
hv.ipython.display(hv.Layout(graphs).cols(2))
df.hvplot() * df.hvplot.scatter()

## Lawyers data set
The Lazega lawyers coworking network partitioned with meta attribute school

In [ ]:
data = np.loadtxt('data/ELwork.dat', dtype=int)
attributeColumns = columns=['seniority', 'status', 'gender', 'office', 'years', 'age', 'practice', 'school']
attributeMap = {
    'status': {1: 'partner', 2: 'associate'},
    'gender': {1: 'man', 2: 'woman'},
    'office': {1: 'Boston', 2: 'Hartford', 3: 'Providence' },
    'practice': {1: 'litigation', 2: 'corporate'},
    'school': {1: 'Harvard / Yale', 2: 'University of Connecticut', 3: 'Other'}
}
attributes = pd.DataFrame(np.loadtxt('data/ELattr.dat', dtype=int), columns=attributeColumns, dtype=int).replace(attributeMap)
attributes['metaData'] = pd.Categorical(attributes['school']).codes
attributes.head()

In [ ]:
G = nx.from_numpy_matrix(data)
for attribute in attributes.columns:
    nx.set_node_attributes(G, attributes[attribute], name=attribute)
G.remove_nodes_from(list(nx.isolates(G)))

In [ ]:
pos = nx.spring_layout(G, seed=5)
metaDataRates = np.arange(0, 2, 0.2)
graphs = []
stats = []
for eta in metaDataRates:
    M, L, Lmeta, Hmeta = findCommunities(G, flags = f"-2 -N1 --meta-data-rate {eta}")
    graphs.append(drawNetwork(G, pos=pos, label=f"{M} modules, eta = {eta:.1f}, Hmeta = {Hmeta:.2f}, L = {L:.2f} bits"))
    stats.append([eta, M, L, L - Lmeta, Lmeta, Hmeta])
df = pd.DataFrame(stats, columns=['eta', 'Number of modules', 'L', 'L - Lmeta', 'Lmeta = eta * Hmeta', 'Hmeta'])
df.set_index('eta', inplace=True)
hv.ipython.display(hv.Layout(graphs).cols(2))
df.hvplot() * df.hvplot.scatter()

#### Compare with codelength disregarding network structure using only meta data as community info

In [ ]:
eta = 1
M, L, Lmeta, Hmeta = findCommunities(G, flags = f"--meta-data-rate {eta}", computeCodelengthOfMetaDataCommunities=True)
drawNetwork(G, pos=pos, label=f"{M} modules, eta = {eta:.1f}, Hmeta = {Hmeta:.2f}, L = {L:.2f} bits")

In [ ]:
import umap

In [ ]:
findCommunities(G, flags = f"-2 -N1 --meta-data-rate {0}")

In [ ]:
M = nx.to_numpy_matrix(G)
w, v = np.linalg.eig(M)
embedding = umap.UMAP(n_neighbors=4, min_dist=0.1, metric='euclidean').fit_transform(v)
posUmap = {i: embedding[i] / 10 for i in range(len(embedding))}


In [ ]:
%%opts Graph(node_size=10, edge_alpha=0.3, node_color='community')
hvnx.draw(G, pos) + hvnx.draw(G, posUmap)